In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!ls /raid/data/DA_BrainDataset/siemens3/df_save.csv

/raid/data/DA_BrainDataset/siemens3/df_save.csv


In [3]:
SIEMENS_PATH = '/raid/data/DA_BrainDataset/siemens3/df_save.csv'
PHILIPS_PATH = '/raid/data/DA_BrainDataset/philips3/df_save.csv'

In [4]:
from torchvision import transforms as TT
from data import SegmentationDataset

transforms = TT.Compose([
    TT.ToTensor()
])


dataset_A = SegmentationDataset(
    dataframe_path = SIEMENS_PATH,
    transform=transforms,
    mask_transform=transforms
)

dataset_B = SegmentationDataset(
    dataframe_path = PHILIPS_PATH,
    transform=transforms,
    mask_transform=transforms
)

len(dataset_A), len(dataset_B)

(10320, 10320)

___


$\mathcal{L}\left(\phi^w ; D^{s \rightarrow t}, D^t\right)=\mathcal{L}_{c e}\left(\phi^w ; D^{s \rightarrow t}\right)+\lambda_{\text {ent }} \mathcal{L}_{\text {ent }}\left(\phi^w ; D^t\right)$ (6)

$\mathcal{L}_{c e}\left(\phi^w ; D^{s \rightarrow t}\right)=-\sum_i\left\langle y_i^s, \log \left(\phi^w\left(x_i^{s \rightarrow t}\right)\right)\right\rangle$.

$\mathcal{L}_{\text {ent }}\left(\phi^w ; D^t\right)=\sum_i \rho\left(-\left\langle\phi^w\left(x_i^t\right), \log \left(\phi^w\left(x_i^t\right)\right)\right\rangle\right)$

$\rho(x)=\left(x^2+0.001^2\right)^\eta$

___

$\begin{aligned} & \mathcal{L}_{s s t}\left(\phi^w ; D^{s \rightarrow t}, \hat{D}^t\right)=\mathcal{L}_{c e}\left(\phi^w ; D^{s \rightarrow t}\right) \\ &+\lambda_{\text {ent }} \mathcal{L}_{\text {ent }}\left(\phi^w ; D^t\right)+\mathcal{L}_{c e}\left(\phi^w ; \hat{D}^t\right)\end{aligned}$ (8)

___

The full training procedure of our FDA semantic
segmentation network consists of:
1. one round of initial training
of M=3 models from scratch using Eq. (6)
2. two more
rounds of self-supervised training using Eq. (8)

In [5]:
import torch

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda', index=0)

In [6]:
import gc

def clear_cache():
    return torch.cuda.empty_cache(), gc.collect()

In [16]:
N_EPOCHS=15
BATCH_SIZE=3

In [17]:
from data import get_train_test_split_loaders

trainA_loader, testA_loader = get_train_test_split_loaders(
    dataset_A, batch_size=BATCH_SIZE, test_size=0.2
)

trainB_loader, testB_loader = get_train_test_split_loaders(
    dataset_B, batch_size=BATCH_SIZE, test_size=0.2
)

In [18]:
# from utils import beta_transform

# img_id = 1

# a = trainA_loader.dataset[img_id]['img']
# b = trainB_loader.dataset[img_id]['img']

# beta_transform(a, b, 0.5, show=True)
# a.shape, b.shape


In [19]:
from models.supervised_models import LitDeepLabV2

clear_cache()

model = LitDeepLabV2(
    num_classes=2,
    n_epochs=N_EPOCHS,
    beta=0.05,
    ent_lambda=0.005,
    eta=2.0
).to(DEVICE)

In [25]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

wb_logger = pl.loggers.WandbLogger(
    name='DeepLab debug',
    project='FDA'
)

checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints',
    monitor='val_miou',
    filename='{epoch:02d}-{val_miou:.3f}',
    mode='max'
)

trainer = pl.Trainer(
    max_epochs=N_EPOCHS,
    logger=wb_logger,
    accelerator='gpu',
    devices=1,
    benchmark=True,
    callbacks=[checkpoint_callback]
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


In [26]:
from pytorch_lightning.trainer.supporters import CombinedLoader
import wandb

trainer.fit(
    model = model,
    train_dataloaders = CombinedLoader({
        'src': trainA_loader,
        'tgt': trainB_loader
    }),
    val_dataloaders = CombinedLoader({
        'src': testA_loader,
        'tgt': testB_loader
    })
)

# wandb.finish()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name       | Type                   | Params
------------------------------------------------------
0 | model      | ResNet101              | 42.6 M
1 | train_miou | MulticlassJaccardIndex | 0     
2 | val_miou   | MulticlassJaccardIndex | 0     
------------------------------------------------------
42.5 M    Trainable params
105 K     Non-trainable params
42.6 M    Total params
170.565   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Exception in thread Thread-29:
Traceback (most recent call last):
  File "/home/v_chernyy/myvenv/lib/python3.9/site-packages/wandb/apis/normalize.py", line 26, in wrapper
    return func(*args, **kwargs)
  File "/home/v_chernyy/myvenv/lib/python3.9/site-packages/wandb/sdk/internal/internal_api.py", line 1692, in upload_urls
    raise CommError(f"Run does not exist {entity}/{project}/{run_id}.")
wandb.errors.CommError: Run does not exist scalyvladimir/FDA/p6kgayle.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/home/v_chernyy/myvenv/lib/python3.9/site-packages/wandb/filesync/upload_job.py", line 67, in run
    success = self.push()
  File "/home/v_chernyy/myvenv/lib/python3.9/site-packages/wandb/filesync/upload_job.py", line 118, in push
    _, upload_headers, result = self._api.upload_urls(project, [self.save_name])
  File "/hom